✊✋🤚 Hand Gesture Controlled Rock-Paper-Scissors Game

In [8]:
import os

# Define the path to the dataset folder
dataset_folder = 'Dataset-rps'  

# Define the subfolders
folders = ['paper', 'rock', 'scissors']

# Loop through each folder and count the number of files
for folder in folders:
    folder_path = os.path.join(dataset_folder, folder)  
    # List all files in the folder and count them
    files = os.listdir(folder_path)
    image_count = len([f for f in files if f.endswith(('jpg', 'png', 'jpeg'))])  
    print(f"Number of images in '{folder}': {image_count}")


Number of images in 'paper': 907
Number of images in 'rock': 907
Number of images in 'scissors': 903


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Paths to your dataset
dataset_folder = 'Dataset-rps'

# Initialize ImageDataGenerator with data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,  
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True, 
    fill_mode='nearest',  
    validation_split=0.2  
)

# Train generator
train_generator = train_datagen.flow_from_directory(
    dataset_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Specify that this is for training
)

# Validation generator (for test data)
validation_generator = train_datagen.flow_from_directory(
    dataset_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Specify that this is for validation/testing
)

# Load pre-trained VGG16 model (without the top fully connected layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for our rock-paper-scissors classification
model = models.Sequential([
    base_model,  # Add the base VGG16 model
    layers.GlobalAveragePooling2D(),  
    layers.Dense(512, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),  
    layers.Dense(3, activation='softmax')  
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with validation data
history = model.fit(
    train_generator,  # Train using the train generator
    epochs=10,  
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # how many bathes per epoch
    validation_data=validation_generator,  # Use the validation generator for testing
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # Validation steps
    verbose=1  # Show progress
)

# Save the trained model
model.save('rps_model.h5')  # Save the trained model for later use




Found 2175 images belonging to 3 classes.
Found 542 images belonging to 3 classes.


D:\Anaconda_\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 416s 6s/step - accuracy: 0.4946 - loss: 1.0031 - val_accuracy: 0.8867 - val_loss: 0.5218
Epoch 2/10
 1/67 ━━━━━━━━━━━━━━━━━━━━ 5:23 5s/step - accuracy: 0.8750 - loss: 0.5122

D:\Anaconda_\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


67/67 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.8750 - loss: 0.5122 - val_accuracy: 0.8770 - val_loss: 0.5206
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 445s 7s/step - accuracy: 0.8092 - loss: 0.5528 - val_accuracy: 0.8125 - val_loss: 0.4378
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 97s 1s/step - accuracy: 0.7500 - loss: 0.5689 - val_accuracy: 0.8301 - val_loss: 0.4370
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 442s 7s/step - accuracy: 0.8474 - loss: 0.4255 - val_accuracy: 0.9121 - val_loss: 0.2811
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.9688 - loss: 0.1755 - val_accuracy: 0.9102 - val_loss: 0.2622
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 426s 6s/step - accuracy: 0.8802 - loss: 0.3355 - val_accuracy: 0.9238 - val_loss: 0.2475
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.9375 - loss: 0.2598 - val_accuracy: 0.9219 - val_loss: 0.2477
Epoch 9/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 505s 8s/step - accuracy: 0.9018 - loss: 0.2808 - val_accuracy: 0.9492 - val_loss: 0.1887

In [10]:
import tensorflow as tf
model = tf.keras.models.load_model('rps_model.h5')
print("Model loaded successfully.")


Model loaded successfully.


In [11]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

import time

# Load CNN model for Paper and Scissors
model = load_model('rps_model.h5', compile=False)
cnn_labels = ['Paper', 'Scissors']  # Only Paper and Scissors for CNN

# MediaPipe setup
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1)

# Check if fingers are folded (for Rock)
def is_rock(hand_landmarks):
    tips = [8, 12, 16, 20]  # Index, Middle, Ring, Pinky fingertips
    folded = 0
    for tip in tips:
        if hand_landmarks.landmark[tip].y > hand_landmarks.landmark[tip - 2].y:
            folded += 1
    return folded == 4  # All fingers folded

cap = cv2.VideoCapture(0)
start_time = time.time()
warmup_time = 5  # 5 seconds initially

while True:
    success, frame = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    frame = cv2.flip(frame, 1)  # Flip frame horizontally for natural mirror effect
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB for MediaPipe
    result = hands.process(img_rgb)

    prediction_text = "No Hand Detected"

    # Check if hand landmarks are detected
    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

            if is_rock(handLms):  # Detect Rock using hand landmarks
                prediction_text = "Rock"
            else:
                # Use CNN for Paper and Scissors
                roi = cv2.resize(frame, (224, 224))  # Resize the frame to match input size of the model
                roi = roi.astype('float32') / 255.0  # Normalize the image
                roi = np.expand_dims(roi, axis=0)  # Add batch dimension

                # Get predictions from the CNN model
                prediction = model.predict(roi)
                class_index = np.argmax(prediction)  # Get index of the class with highest probability

                if class_index == 0:  # Paper predicted
                    prediction_text = "Paper"
                else:  # If it's not Paper, then it must be Scissors
                    prediction_text = "Scissors"

    # Display the prediction
    cv2.putText(frame, f"Prediction: {prediction_text}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the frame with landmarks and prediction text
    cv2.imshow("Rock-Paper-Scissors", frame)

    # Exit on ESC key
    if cv2.waitKey(1) & 0xFF == 27:
        break

    # After the first run, reduce the warmup time
    if time.time() - start_time > warmup_time:
        warmup_time = 2

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import time
import random

# Load CNN model for Paper and Scissors
model = load_model('rps_model.h5', compile=False)
cnn_labels = ['Paper', 'Scissors']  # Only Paper and Scissors for CNN

# MediaPipe setup
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1)

# Check if fingers are folded (for Rock)
def is_rock(hand_landmarks):
    tips = [8, 12, 16, 20]  # Index, Middle, Ring, Pinky fingertips
    folded = 0
    for tip in tips:
        if hand_landmarks.landmark[tip].y > hand_landmarks.landmark[tip - 2].y:
            folded += 1
    return folded == 4  # All fingers folded

# Computer's choice logic
def computer_choice():
    return random.choice(['Rock', 'Paper', 'Scissors'])

# Game result logic
def get_winner(player, computer):
    if player == computer:
        return 'Draw'
    elif (player == 'Rock' and computer == 'Scissors') or \
         (player == 'Paper' and computer == 'Rock') or \
         (player == 'Scissors' and computer == 'Paper'):
        return 'Player'
    else:
        return 'Computer'

# Initialize scores
player_score = 0
computer_score = 0

cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Set width to 1280
cap.set(4, 720)   # Set height to 720
start_time = time.time()
warmup_time = 5  # Initial warmup time before the game starts

while True:
    # Countdown before the game starts (2 seconds)
    for i in range(2, 0, -1):
        frame = np.zeros((720, 1280, 3), dtype=np.uint8)
        cv2.putText(frame, f"Get Ready! {i}", (500, 360), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 4)
        cv2.imshow("Rock-Paper-Scissors Game", frame)
        if cv2.waitKey(1000) & 0xFF == 27:
            cap.release()
            cv2.destroyAllWindows()
            exit()

    # Start round timer
    round_start = time.time()
    player_choice = "None"
    prediction_text = "Detecting..."

    while time.time() - round_start < 2:  # 2-second round timer
        success, frame = cap.read()
        if not success:
            print("Failed to grab frame")
            break

        frame = cv2.flip(frame, 1)  # Flip frame horizontally for natural mirror effect
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB for MediaPipe
        result = hands.process(img_rgb)

        # Check for hand landmarks
        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

                if is_rock(handLms):  # Detect Rock using hand landmarks
                    prediction_text = "Rock"
                else:
                    # Use CNN for Paper and Scissors
                    roi = cv2.resize(frame, (224, 224))  # Resize the frame to match input size of the model
                    roi = roi.astype('float32') / 255.0  # Normalize the image
                    roi = np.expand_dims(roi, axis=0)  # Add batch dimension

                    # Get predictions from the CNN model
                    prediction = model.predict(roi)
                    class_index = np.argmax(prediction)  # Get index of the class with highest probability

                    if class_index == 0:  # Paper predicted
                        prediction_text = "Paper"
                    else:  # If it's not Paper, then it must be Scissors
                        prediction_text = "Scissors"

        # Display the round countdown and prediction
        cv2.putText(frame, f"Detecting... {prediction_text}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        remaining_time = 2 - int(time.time() - round_start)
        # Move "Time Left" text to a more left position
        cv2.putText(frame, f"Time Left: {remaining_time}s", (900, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow("Rock-Paper-Scissors Game", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            cap.release()
            cv2.destroyAllWindows()
            exit()

    # Get computer's choice and calculate winner
    computer_gesture = computer_choice()
    winner = get_winner(prediction_text, computer_gesture)
    
    if winner == 'Player':
        player_score += 1
    elif winner == 'Computer':
        computer_score += 1

    # Show round result
    result_frame = np.zeros((720, 1280, 3), dtype=np.uint8)
    cv2.putText(result_frame, f"You: {prediction_text}", (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)
    cv2.putText(result_frame, f"Computer: {computer_gesture}", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 255), 3)
    
    # Use if-elif-else to handle the winner message
    if winner == 'Player':
        cv2.putText(result_frame, f"Winner: You", (50, 280), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
    elif winner == 'Computer':
        cv2.putText(result_frame, f"Winner: Computer", (50, 280), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
    else:
        cv2.putText(result_frame, f"It's a Draw!", (50, 280), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 4)
    
    cv2.putText(result_frame, f"Score - You: {player_score} | Computer: {computer_score}", (20, 360),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow("Rock-Paper-Scissors Game", result_frame)
    cv2.waitKey(3000)


    # Check if anyone reached 5 points
    if player_score == 5 or computer_score == 5:
        final_frame = np.zeros((720, 1280, 3), dtype=np.uint8)
        winner_text = "You Win!" if player_score == 5 else "Computer Wins!"
        cv2.putText(final_frame, winner_text, (500, 250), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 0, 255), 5)
        cv2.putText(final_frame, "Press ESC to exit", (400, 320), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
        cv2.imshow("Rock-Paper-Scissors Game", final_frame)
        if cv2.waitKey(0) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 